# Import

In [11]:
import numpy as np
import pandas as pd

# Data preporation

In [12]:
# Load data of previous day
file_name = 'release/covid19-russia-cases-scrf.csv'
rus_df = pd.read_csv(file_name)
rus_df.tail()

,Date,Region/City,Region/City-Eng,Region_ID,Day-Confirmed,Day-Deaths,Day-Recovered,Confirmed,Deaths,Recovered
9096,2020-06-26,Челябинская область,Chelyabinsk region,74.0,160.0,18.0,0.0,6602.0,88.0,3500.0
9097,2020-06-26,Чеченская Республика,Chechen Republic,95.0,10.0,0.0,1.0,1664.0,21.0,1177.0
9098,2020-06-26,Чукотский автономный округ,Chukotka Autonomous Okrug,87.0,1.0,0.0,1.0,136.0,1.0,123.0
9099,2020-06-26,Ямало-Ненецкий АО,Yamalo-Nenets Autonomous Okrug,89.0,149.0,0.0,52.0,4858.0,34.0,2058.0
9100,2020-06-26,Ярославская область,Yaroslavl region,76.0,43.0,0.0,47.0,4614.0,18.0,2950.0


In [13]:
# Read update data
# Result of data_parsing_SCRF.ipynb
file_name = f'covid19-russia-cases-upd-scrf.csv'
upd_df = pd.read_csv(file_name)
upd_df.sort_values('Confirmed', ascending=False).head()

,Region/City,Confirmed,Recovered,Deaths,Date,Day-Confirmed,Day-Deaths,Day-Recovered
29,Москва,219354,146043,3714,2020-06-27,750.0,20.0,1852.0
30,Московская область,56609,31912,858,2020-06-27,366.0,12.0,1421.0
64,Санкт-Петербург,23518,16211,1065,2020-06-27,224.0,57.0,716.0
33,Нижегородская область,16838,10400,223,2020-06-27,214.0,8.0,456.0
67,Свердловская область,12093,7009,90,2020-06-27,216.0,5.0,223.0


In [14]:
# Strip text data
rus_df['Region/City'] = rus_df['Region/City'].astype('str').str.strip('\u200b') 
upd_df['Region/City'] = upd_df['Region/City'].astype('str').str.strip('\u200b')

# Date to datetime
upd_df['Date'] = pd.to_datetime(upd_df['Date'])
rus_df['Date'] = pd.to_datetime(rus_df['Date'])

# Updating

In [15]:
# Add region ID and Eng columns
regions_df = pd.read_csv('release/regions-info.csv')

right = regions_df[['Region_ID', 'Region', 'Region_eng']]
upd_df = upd_df.merge(right, left_on='Region/City', right_on='Region', how='left')
upd_df['Region/City-Eng'] = upd_df.Region_eng
upd_df = upd_df[['Date', 'Region/City', 'Region/City-Eng', 'Region_ID', 'Day-Confirmed', 'Day-Deaths', 'Day-Recovered', 'Confirmed', 'Deaths', 'Recovered']]

In [16]:
rus_df = pd.concat([rus_df, upd_df]).reset_index(drop=True)
rus_df.tail(10)

,Date,Region/City,Region/City-Eng,Region_ID,Day-Confirmed,Day-Deaths,Day-Recovered,Confirmed,Deaths,Recovered
9176,2020-06-27,Тюменская область,Tyumen region,72.0,59.0,0.0,47.0,2886.0,16.0,1731.0
9177,2020-06-27,Удмуртская Республика,Udmurt republic,18.0,23.0,1.0,29.0,1357.0,16.0,998.0
9178,2020-06-27,Ульяновская область,Ulyanovsk region,73.0,109.0,1.0,130.0,5461.0,36.0,2840.0
9179,2020-06-27,Хабаровский край,Khabarovsk region,27.0,79.0,0.0,15.0,3978.0,28.0,2063.0
9180,2020-06-27,Ханты-Мансийский АО,Khanty-Mansiysk,86.0,280.0,3.0,229.0,8117.0,44.0,4091.0
9181,2020-06-27,Челябинская область,Chelyabinsk region,74.0,158.0,0.0,60.0,6760.0,88.0,3560.0
9182,2020-06-27,Чеченская Республика,Chechen Republic,95.0,13.0,0.0,3.0,1677.0,21.0,1180.0
9183,2020-06-27,Чукотский автономный округ,Chukotka Autonomous Okrug,87.0,2.0,0.0,2.0,138.0,1.0,125.0
9184,2020-06-27,Ямало-Ненецкий АО,Yamalo-Nenets Autonomous Okrug,89.0,147.0,0.0,95.0,5005.0,34.0,2153.0
9185,2020-06-27,Ярославская область,Yaroslavl region,76.0,44.0,1.0,46.0,4658.0,19.0,2996.0


In [17]:
# Negative values fix
# Negative values can appear if some cases was removed from statistic
def negval_fix(reg):
    last = reg.iloc[-1]
    diff_c = last['Day-Confirmed']*(-1) if last['Day-Confirmed'] < 0 else 0
    diff_d = last['Day-Deaths']*(-1) if last['Day-Deaths'] < 0 else 0
    diff_r = last['Day-Recovered']*(-1) if last['Day-Recovered'] < 0 else 0
    
    last['Day-Confirmed'] += diff_c
    last['Day-Deaths']    += diff_d
    last['Day-Recovered'] += diff_r
    reg.iloc[-1] = last
    
    if diff_c + diff_d + diff_r > 0:
        print(diff_c, diff_d, diff_r, last['Region/City'])
        prev = reg.iloc[-2]
        prev['Confirmed'] -= diff_c
        prev['Deaths']    -= diff_d
        prev['Recovered'] -= diff_r
        reg.iloc[-2] = prev
    
    return reg
    
rus_df = rus_df.groupby('Region/City').apply(negval_fix).reset_index(drop=True)

In [18]:
# Manual fixes if needed
# rus_df.loc[287, 'Confirmed'] = 5
# rus_df.loc[321, 'Confirmed'] = 5
# rus_df.loc[359, 'Day-Confirmed'] = 0

# rus_df.loc[3775, 'Deaths'] = 7
# rus_df.loc[3775, 'Day-Deaths'] = 0

# Data checking

In [19]:
rus_sum = rus_df.loc[rus_df['Region/City'] != 'Diamond Princess']

print('Russia Confirmed:', rus_sum.groupby('Region/City')['Confirmed'].max().sum(),
      '| By day:', rus_sum.groupby('Region/City')['Day-Confirmed'].sum().sum())
print('Russia Deaths:', rus_sum.groupby('Region/City')['Deaths'].max().sum(),
      '| By day:', rus_sum.groupby('Region/City')['Day-Deaths'].sum().sum())
print('Russia Recovered:', rus_sum.groupby('Region/City')['Recovered'].max().sum(),
      '| By day:', rus_sum.groupby('Region/City')['Day-Recovered'].sum().sum())

rus_regs = rus_sum.groupby('Region/City')['Confirmed'].max().reset_index()
rus_regs['Confirmed-ByDay'] = rus_sum.groupby('Region/City')['Day-Confirmed'].sum().reset_index(drop=True)
print(rus_regs.loc[rus_regs['Confirmed'] != rus_regs['Confirmed-ByDay']])

rus_regs = rus_sum.groupby('Region/City')['Recovered'].max().reset_index()
rus_regs['Recovered-ByDay'] = rus_sum.groupby('Region/City')['Day-Recovered'].sum().reset_index(drop=True)
print(rus_regs.loc[rus_regs['Recovered'] != rus_regs['Recovered-ByDay']])

rus_regs = rus_sum.groupby('Region/City')['Deaths'].max().reset_index()
rus_regs['Deaths-ByDay'] = rus_sum.groupby('Region/City')['Day-Deaths'].sum().reset_index(drop=True)
print(rus_regs.loc[rus_regs['Deaths'] != rus_regs['Deaths-ByDay']])

# rus_regs.to_csv('rus_regs.csv')

Russia Confirmed: 627646.0 | By day: 627646.0
Russia Deaths: 8969.0 | By day: 8969.0
Russia Recovered: 393352.0 | By day: 393352.0
Empty DataFrame
Columns: [Region/City, Confirmed, Confirmed-ByDay]
Index: []
Empty DataFrame
Columns: [Region/City, Recovered, Recovered-ByDay]
Index: []
Empty DataFrame
Columns: [Region/City, Deaths, Deaths-ByDay]
Index: []


# Data saving

In [20]:
file_name = 'release/covid19-russia-cases-scrf.csv'
rus_df.to_csv(file_name, index=False)